# 🎯 Módulo 4: Desvendando os Tokens - O DNA das LLMs!

**Curso: Introdução à LLMs**

*By Pedro Nunes Guth*

---

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-04_img_01.png)

E aí, pessoal! Bora mergulhar no mundo dos **tokens**? 🚀

Lembra quando falamos sobre a arquitetura Transformer no módulo anterior? Pois é, chegou a hora de entender como as LLMs realmente "enxergam" o texto que você manda pra elas!

Tá, mas o que é um token afinal? 🤔

## 🧠 O que são Tokens?

**Token** é basicamente a "moeda" que as LLMs usam pra processar texto. Imagina que você tem um sanduíche gigante e precisa cortar ele em pedaços menores pra conseguir comer. Os tokens são esses pedaços!

### Por que não processar palavra por palavra?

Pensa comigo:
- E se eu escrever "anti-inflamatório"? São 2 palavras ou 1?
- E "João"? Como a IA vai saber que é um nome?
- E emojis? 😎🔥

Por isso a tokenização é **FUNDAMENTAL** pras LLMs!

### Tipos de Tokens:
1. **Subword tokens** - Pedaços de palavras
2. **Word tokens** - Palavras completas
3. **Character tokens** - Caracteres individuais
4. **Tokens especiais** - [CLS], [SEP], etc.

**Dica do Pedro:** A maioria das LLMs modernas usa subword tokenization. É tipo cortar o sanduíche em pedaços que fazem sentido! 🥪

In [ ]:
# Vamos começar com os imports necessários!
# Bora instalar e importar as bibliotecas que vamos usar

!pip install transformers tiktoken matplotlib seaborn wordcloud

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import json
from wordcloud import WordCloud

# Configurações visuais
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("📚 Bibliotecas carregadas com sucesso!")
print("🚀 Bora tokenizar!")

## 🔄 Processo de Tokenização

A tokenização acontece em algumas etapas. Vou explicar usando uma analogia bem brasileira:

```mermaid
graph TD
    A["Texto: 'Vai, Brasil! 🇧🇷'"] --> B[Pré-processamento]
    B --> C[Normalização]
    C --> D[Aplicar Algoritmo]
    D --> E["Tokens: ['Vai', ',', 'Brasil', '!', '🇧🇷']"]
    E --> F[IDs Numéricos]
    F --> G["[1234, 45, 789, 23, 9876]"]
```

### Etapas da Tokenização:

1. **Pré-processamento**: Limpar o texto
2. **Normalização**: Converter maiúsculas, remover acentos (às vezes)
3. **Aplicar algoritmo**: BPE, WordPiece, SentencePiece
4. **Converter para IDs**: Cada token vira um número

**Importante**: Cada modelo tem seu próprio vocabulário de tokens!

In [ ]:
# Vamos criar um tokenizador simples pra entender o conceito

class TokenizadorSimples:
    def __init__(self):
        self.vocab = {}
        self.token_to_id = {}
        self.id_to_token = {}
        
    def treinar(self, textos):
        """Treina o tokenizador com uma lista de textos"""
        print("🏋️ Treinando o tokenizador...")
        
        # Coleta todos os caracteres únicos
        chars = set()
        for texto in textos:
            chars.update(list(texto.lower()))
        
        # Cria vocabulário
        tokens_especiais = ['<pad>', '<unk>', '<start>', '<end>']
        self.vocab = tokens_especiais + sorted(list(chars))
        
        # Cria mapeamentos
        self.token_to_id = {token: i for i, token in enumerate(self.vocab)}
        self.id_to_token = {i: token for i, token in enumerate(self.vocab)}
        
        print(f"✅ Vocabulário criado com {len(self.vocab)} tokens!")
        print(f"📋 Primeiros 10 tokens: {self.vocab[:10]}")
        
    def tokenizar(self, texto):
        """Tokeniza um texto em caracteres"""
        tokens = []
        for char in texto.lower():
            if char in self.token_to_id:
                tokens.append(char)
            else:
                tokens.append('<unk>')  # Token desconhecido
        return tokens
    
    def encode(self, texto):
        """Converte texto em IDs"""
        tokens = self.tokenizar(texto)
        return [self.token_to_id[token] for token in tokens]
    
    def decode(self, ids):
        """Converte IDs de volta para texto"""
        tokens = [self.id_to_token[id] for id in ids]
        return ''.join(tokens)

# Teste do nosso tokenizador
textos_treino = [
    "Olá, mundo!",
    "Python é incrível",
    "Inteligência Artificial",
    "Tokens são importantes"
]

tokenizador = TokenizadorSimples()
tokenizador.treinar(textos_treino)

# Testando
texto_teste = "Olá, IA!"
print(f"\n🔍 Texto original: '{texto_teste}'")
print(f"🎯 Tokens: {tokenizador.tokenizar(texto_teste)}")
print(f"🔢 IDs: {tokenizador.encode(texto_teste)}")
print(f"🔄 Decodificado: '{tokenizador.decode(tokenizador.encode(texto_teste))}'")

## 🧬 Algoritmos de Tokenização

Agora vamos falar dos algoritmos que as LLMs de verdade usam! São mais sofisticados que o nosso exemplo anterior.

### 1. **Byte Pair Encoding (BPE)**
Usado pelo GPT! Funciona assim:
- Começa com caracteres individuais
- Vai juntando os pares mais frequentes
- Tipo montar palavras com Lego! 🧱

### 2. **WordPiece**
Usado pelo BERT! Similar ao BPE, mas:
- Usa probabilidade pra decidir as junções
- Prefere subwords que maximizam a verossimilhança

### 3. **SentencePiece**
Usado pelo T5! O mais moderno:
- Trata o texto como sequência de bytes
- Não precisa de pré-processamento
- Funciona com qualquer idioma!

**Dica do Pedro:** Cada algoritmo tem suas vantagens. O SentencePiece é tipo o "canivete suíço" da tokenização! 🔧

In [ ]:
# Vamos implementar um BPE simplificado pra entender o conceito!

class BPESimplificado:
    def __init__(self, num_merges=100):
        self.num_merges = num_merges
        self.vocab = set()
        self.merges = []
        
    def get_stats(self, vocab):
        """Conta frequência de pares consecutivos"""
        pairs = {}
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                pairs[pair] = pairs.get(pair, 0) + freq
        return pairs
    
    def merge_vocab(self, pair, v_in):
        """Faz o merge de um par no vocabulário"""
        v_out = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        
        for word in v_in:
            new_word = word.replace(bigram, replacement)
            v_out[new_word] = v_in[word]
        return v_out
    
    def treinar(self, textos):
        """Treina o BPE"""
        print("🚀 Iniciando treinamento BPE...")
        
        # Prepara vocabulário inicial (palavras como caracteres separados)
        vocab = {}
        for texto in textos:
            palavras = texto.lower().split()
            for palavra in palavras:
                # Adiciona espaço entre caracteres + </w> no final
                palavra_tokenizada = ' '.join(list(palavra)) + ' </w>'
                vocab[palavra_tokenizada] = vocab.get(palavra_tokenizada, 0) + 1
        
        print(f"📊 Vocabulário inicial: {len(vocab)} palavras")
        
        # Executa merges
        for i in range(self.num_merges):
            pairs = self.get_stats(vocab)
            if not pairs:
                break
                
            # Pega o par mais frequente
            best_pair = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best_pair, vocab)
            self.merges.append(best_pair)
            
            if i % 20 == 0:
                print(f"🔄 Merge {i}: {best_pair} (freq: {pairs[best_pair]})")
        
        # Extrai vocabulário final
        self.vocab = set()
        for word in vocab:
            self.vocab.update(word.split())
            
        print(f"✅ Treinamento concluído!")
        print(f"📈 Vocabulário final: {len(self.vocab)} tokens")
        print(f"🔧 Merges realizados: {len(self.merges)}")
        
        return self.vocab, self.merges

# Teste do BPE
textos_bpe = [
    "inteligência artificial é incrível",
    "processamento de linguagem natural",
    "transformers revolucionaram a área",
    "tokenização é fundamental",
    "artificial intelligence rocks"
]

bpe = BPESimplificado(num_merges=50)
vocab_final, merges = bpe.treinar(textos_bpe)

print(f"\n🎯 Alguns tokens aprendidos:")
for i, token in enumerate(sorted(list(vocab_final))[:15]):
    print(f"  {i+1}. '{token}'")

In [ ]:
# Vamos visualizar como diferentes tokenizadores funcionam na prática!

from transformers import AutoTokenizer
import tiktoken

# Carrega diferentes tokenizadores
print("📥 Carregando tokenizadores famosos...")

# BERT (WordPiece)
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# GPT-2 (BPE)
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')

# GPT-4 (tiktoken)
try:
    gpt4_tokenizer = tiktoken.encoding_for_model("gpt-4")
except:
    gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")  # Fallback

print("✅ Tokenizadores carregados!")

# Texto de teste em português
texto_br = "Inteligência artificial está revolucionando o Brasil! 🇧🇷"

print(f"\n🎯 Texto de teste: '{texto_br}'\n")

# Compara tokenizações
print("🔍 BERT (WordPiece):")
bert_tokens = bert_tokenizer.tokenize(texto_br)
print(f"  Tokens: {bert_tokens}")
print(f"  Quantidade: {len(bert_tokens)} tokens")

print("\n🔍 GPT-2 (BPE):")
gpt2_tokens = gpt2_tokenizer.tokenize(texto_br)
print(f"  Tokens: {gpt2_tokens}")
print(f"  Quantidade: {len(gpt2_tokens)} tokens")

print("\n🔍 GPT-4 (tiktoken):")
gpt4_tokens = [gpt4_tokenizer.decode([token]) for token in gpt4_tokenizer.encode(texto_br)]
print(f"  Tokens: {gpt4_tokens}")
print(f"  Quantidade: {len(gpt4_tokens)} tokens")

# Análise dos resultados
print("\n📊 Análise:")
print(f"  BERT quebrou em {len(bert_tokens)} pedaços")
print(f"  GPT-2 quebrou em {len(gpt2_tokens)} pedaços")
print(f"  GPT-4 quebrou em {len(gpt4_tokens)} pedaços")
print("\n💡 Cada modelo 'vê' o texto de forma diferente!")

## 📊 Visualizando a Tokenização

Vamos criar algumas visualizações massa pra entender melhor como funciona!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-04_img_02.png)

**Dica do Pedro:** Visualizar dados é sempre uma boa! Ajuda a "ver" o que tá rolando nos algoritmos. 📈

In [ ]:
# Vamos criar visualizações incríveis sobre tokenização!

# Preparando dados para comparação
textos_teste = [
    "Olá mundo",
    "Inteligência artificial", 
    "Processamento de linguagem natural",
    "Transformers são revolucionários",
    "Machine learning é incrível",
    "Deep learning mudou tudo"
]

# Coleta dados de tokenização
resultados = []
for texto in textos_teste:
    bert_count = len(bert_tokenizer.tokenize(texto))
    gpt2_count = len(gpt2_tokenizer.tokenize(texto))
    gpt4_count = len(gpt4_tokenizer.encode(texto))
    
    resultados.append({
        'texto': texto,
        'BERT': bert_count,
        'GPT-2': gpt2_count,
        'GPT-4': gpt4_count
    })

# Gráfico de barras comparativo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Comparação de tokenizadores
textos_curtos = [t[:15] + '...' if len(t) > 15 else t for t in textos_teste]
bert_counts = [r['BERT'] for r in resultados]
gpt2_counts = [r['GPT-2'] for r in resultados]
gpt4_counts = [r['GPT-4'] for r in resultados]

x = np.arange(len(textos_curtos))
width = 0.25

ax1.bar(x - width, bert_counts, width, label='BERT (WordPiece)', color='#FF6B6B', alpha=0.8)
ax1.bar(x, gpt2_counts, width, label='GPT-2 (BPE)', color='#4ECDC4', alpha=0.8)
ax1.bar(x + width, gpt4_counts, width, label='GPT-4 (tiktoken)', color='#45B7D1', alpha=0.8)

ax1.set_xlabel('Textos de Teste')
ax1.set_ylabel('Número de Tokens')
ax1.set_title('🔍 Comparação de Tokenizadores')
ax1.set_xticks(x)
ax1.set_xticklabels(textos_curtos, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Subplot 2: Eficiência por caractere
character_counts = [len(texto) for texto in textos_teste]
bert_efficiency = [bert_counts[i]/character_counts[i] for i in range(len(textos_teste))]
gpt2_efficiency = [gpt2_counts[i]/character_counts[i] for i in range(len(textos_teste))]
gpt4_efficiency = [gpt4_counts[i]/character_counts[i] for i in range(len(textos_teste))]

ax2.plot(character_counts, bert_efficiency, 'o-', label='BERT', linewidth=2, markersize=8)
ax2.plot(character_counts, gpt2_efficiency, 's-', label='GPT-2', linewidth=2, markersize=8)
ax2.plot(character_counts, gpt4_efficiency, '^-', label='GPT-4', linewidth=2, markersize=8)

ax2.set_xlabel('Número de Caracteres')
ax2.set_ylabel('Tokens por Caractere')
ax2.set_title('📈 Eficiência da Tokenização')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 Insights das visualizações:")
print("  • Cada tokenizador tem comportamento diferente")
print("  • Textos maiores geralmente são mais eficientes")
print("  • GPT-4 tende a ser mais eficiente (menos tokens)")

In [ ]:
# Vamos criar uma nuvem de palavras dos tokens mais comuns!

# Analisa vocabulário do GPT-2
print("🔍 Analisando vocabulário do GPT-2...")

# Pega alguns tokens do vocabulário
vocab_gpt2 = gpt2_tokenizer.get_vocab()
token_ids = list(range(0, min(1000, len(vocab_gpt2))))  # Primeiros 1000 tokens
tokens_sample = [gpt2_tokenizer.decode([id]) for id in token_ids]

# Remove tokens especiais e muito pequenos
tokens_limpos = []
for token in tokens_sample:
    token_clean = token.strip()
    if len(token_clean) > 1 and token_clean.isalpha():
        tokens_limpos.append(token_clean)

# Cria frequências artificiais (em um caso real, usaríamos frequências do corpus)
np.random.seed(42)
frequencias = {token: np.random.randint(10, 100) for token in tokens_limpos[:100]}

# Gera nuvem de palavras
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Nuvem de palavras
wordcloud = WordCloud(width=400, height=400, 
                     background_color='white',
                     colormap='viridis',
                     max_words=50).generate_from_frequencies(frequencias)

ax1.imshow(wordcloud, interpolation='bilinear')
ax1.axis('off')
ax1.set_title('☁️ Nuvem de Tokens GPT-2', fontsize=16, pad=20)

# Gráfico de barras dos mais frequentes
top_tokens = sorted(frequencias.items(), key=lambda x: x[1], reverse=True)[:15]
tokens_top = [item[0] for item in top_tokens]
freqs_top = [item[1] for item in top_tokens]

bars = ax2.barh(tokens_top, freqs_top, color=plt.cm.viridis(np.linspace(0, 1, len(tokens_top))))
ax2.set_xlabel('Frequência (simulada)')
ax2.set_title('📊 Top 15 Tokens Mais Frequentes')
ax2.grid(True, alpha=0.3)

# Adiciona valores nas barras
for i, (bar, freq) in enumerate(zip(bars, freqs_top)):
    ax2.text(freq + 1, i, str(freq), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📈 Estatísticas do vocabulário GPT-2:")
print(f"  • Tamanho total: {len(vocab_gpt2):,} tokens")
print(f"  • Tokens analisados: {len(tokens_limpos)}")
print(f"  • Token mais 'frequente': {top_tokens[0][0]}")

## ⚡ Tokens Especiais

Tá, mas além das palavras normais, existem tokens especiais que são **SUPER** importantes!

### Principais Tokens Especiais:

- **`<pad>`** - Padding (preenchimento)
- **`<unk>`** - Unknown (desconhecido) 
- **`<start>/<s>`** - Início de sequência
- **`<end>/</s>`** - Fim de sequência
- **`<mask>`** - Mascaramento (BERT)
- **`<cls>`** - Classificação
- **`<sep>`** - Separador

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-04_img_03.png)

### Por que são importantes?

1. **Estrutura**: Definem início/fim de textos
2. **Padding**: Fazem sequências ficarem do mesmo tamanho
3. **Mascaramento**: Permitem o treinamento do BERT
4. **Separação**: Dividem diferentes partes do input

**Dica do Pedro:** Os tokens especiais são tipo a "pontuação" do mundo das LLMs. Sem eles, seria bagunça total! 🎭

In [ ]:
# Vamos explorar os tokens especiais dos diferentes modelos!

print("🎭 Explorando tokens especiais...\n")

# BERT tokens especiais
print("🔍 BERT (WordPiece):")
bert_especiais = {
    'PAD': bert_tokenizer.pad_token,
    'UNK': bert_tokenizer.unk_token, 
    'CLS': bert_tokenizer.cls_token,
    'SEP': bert_tokenizer.sep_token,
    'MASK': bert_tokenizer.mask_token
}

for nome, token in bert_especiais.items():
    if token:
        token_id = bert_tokenizer.convert_tokens_to_ids(token)
        print(f"  {nome}: '{token}' (ID: {token_id})")

# GPT-2 tokens especiais  
print("\n🔍 GPT-2 (BPE):")
gpt2_especiais = {
    'PAD': gpt2_tokenizer.pad_token,
    'UNK': gpt2_tokenizer.unk_token,
    'BOS': gpt2_tokenizer.bos_token,
    'EOS': gpt2_tokenizer.eos_token
}

for nome, token in gpt2_especiais.items():
    if token:
        token_id = gpt2_tokenizer.convert_tokens_to_ids(token)
        print(f"  {nome}: '{token}' (ID: {token_id})")
    else:
        print(f"  {nome}: None (não definido)")

# Exemplo prático: como o BERT processa uma frase
print("\n🛠️ Exemplo prático - BERT processando:")
frase = "Olá! Como vai?"
print(f"\nTexto: '{frase}'")

# Tokenização completa do BERT
tokens_bert = bert_tokenizer.tokenize(frase)
print(f"Tokens: {tokens_bert}")

# Com tokens especiais
input_ids = bert_tokenizer.encode(frase)
tokens_completos = bert_tokenizer.convert_ids_to_tokens(input_ids)
print(f"Com especiais: {tokens_completos}")
print(f"IDs: {input_ids}")

# Explicação
print("\n💡 O que aconteceu:")
print(f"  • [CLS] foi adicionado no início (ID: {input_ids[0]})")
print(f"  • [SEP] foi adicionado no final (ID: {input_ids[-1]})")
print(f"  • Total de tokens: {len(input_ids)}")

In [ ]:
# Vamos simular como funciona o padding com tokens especiais!

print("🎯 Simulando padding com tokens especiais\n")

# Várias frases de tamanhos diferentes
frases = [
    "Oi!",
    "Como você está hoje?", 
    "Inteligência artificial é incrível e está mudando o mundo",
    "Tokens"
]

# Tokeniza todas as frases
tokens_por_frase = []
ids_por_frase = []

for i, frase in enumerate(frases):
    tokens = bert_tokenizer.tokenize(frase)
    ids = bert_tokenizer.encode(frase, add_special_tokens=True)
    tokens_por_frase.append(tokens)
    ids_por_frase.append(ids)
    print(f"Frase {i+1}: '{frase}'")
    print(f"  Tokens: {tokens}")
    print(f"  IDs: {ids}")
    print(f"  Comprimento: {len(ids)} tokens\n")

# Encontra o tamanho máximo
max_length = max(len(ids) for ids in ids_por_frase)
print(f"📏 Tamanho máximo necessário: {max_length} tokens")

# Aplica padding
print("\n🔧 Aplicando padding:")
ids_padded = []
attention_masks = []

for i, ids in enumerate(ids_por_frase):
    # Padding
    padding_length = max_length - len(ids)
    ids_pad = ids + [bert_tokenizer.pad_token_id] * padding_length
    
    # Attention mask (1 para tokens reais, 0 para padding)
    attention_mask = [1] * len(ids) + [0] * padding_length
    
    ids_padded.append(ids_pad)
    attention_masks.append(attention_mask)
    
    print(f"\nFrase {i+1} (padded):")
    print(f"  IDs: {ids_pad}")
    print(f"  Mask: {attention_mask}")
    print(f"  Comprimento: {len(ids_pad)} tokens")

print("\n✅ Agora todas as frases têm o mesmo tamanho!")
print("💡 O modelo usa a attention mask para ignorar os tokens de padding")

## 🎨 Visualizando o Processo Completo

Vamos criar um diagrama que mostra todo o pipeline de tokenização!

```mermaid
graph TD
    A["📝 Texto Original<br/>Inteligência Artificial"] --> B["🧹 Pré-processamento<br/>Limpar, normalizar"]
    B --> C["⚡ Tokenização<br/>WordPiece/BPE/SentencePiece"]
    C --> D["🏷️ Adicionar Tokens Especiais<br/>[CLS] + tokens + [SEP]"]
    D --> E["🔢 Conversão para IDs<br/>[101, 1234, 5678, 102]"]
    E --> F["📏 Padding<br/>Mesmo tamanho"]
    F --> G["🎭 Attention Masks<br/>1s e 0s"]
    G --> H["🚀 Pronto para o Modelo!"]
```

**Dica do Pedro:** Esse pipeline é o coração do processamento de texto nas LLMs! Cada etapa é crucial. 💖

In [ ]:
# Vamos criar uma visualização do pipeline completo!

def visualizar_pipeline_tokenizacao(texto, tokenizer, modelo_nome):
    """Visualiza todo o pipeline de tokenização"""
    print(f"\n🎬 Pipeline de Tokenização - {modelo_nome}")
    print("=" * 50)
    
    # Etapa 1: Texto original
    print(f"\n1️⃣ Texto Original:")
    print(f"   '{texto}'")
    print(f"   📊 {len(texto)} caracteres")
    
    # Etapa 2: Tokenização básica
    print(f"\n2️⃣ Tokenização:")
    tokens = tokenizer.tokenize(texto)
    print(f"   Tokens: {tokens}")
    print(f"   📊 {len(tokens)} tokens")
    
    # Etapa 3: Com tokens especiais
    print(f"\n3️⃣ Com Tokens Especiais:")
    input_ids = tokenizer.encode(texto, add_special_tokens=True)
    tokens_completos = tokenizer.convert_ids_to_tokens(input_ids)
    print(f"   Tokens: {tokens_completos}")
    print(f"   📊 {len(tokens_completos)} tokens")
    
    # Etapa 4: IDs numéricos
    print(f"\n4️⃣ IDs Numéricos:")
    print(f"   {input_ids}")
    
    # Etapa 5: Com padding (simulado para tamanho 15)
    print(f"\n5️⃣ Com Padding (max_length=15):")
    if len(input_ids) < 15:
        padded_ids = input_ids + [tokenizer.pad_token_id] * (15 - len(input_ids))
        attention_mask = [1] * len(input_ids) + [0] * (15 - len(input_ids))
    else:
        padded_ids = input_ids[:15]
        attention_mask = [1] * 15
    
    print(f"   IDs: {padded_ids}")
    print(f"   Mask: {attention_mask}")
    
    return {
        'tokens': tokens,
        'tokens_completos': tokens_completos,
        'input_ids': input_ids,
        'padded_ids': padded_ids,
        'attention_mask': attention_mask
    }

# Testa com diferentes modelos
texto_exemplo = "Transformers são revolucionários!"

# BERT
result_bert = visualizar_pipeline_tokenizacao(texto_exemplo, bert_tokenizer, "BERT")

# GPT-2 
result_gpt2 = visualizar_pipeline_tokenizacao(texto_exemplo, gpt2_tokenizer, "GPT-2")

print("\n🏆 Resumo Comparativo:")
print(f"  BERT: {len(result_bert['input_ids'])} tokens")
print(f"  GPT-2: {len(result_gpt2['input_ids'])} tokens")
print("\n💡 Diferentes modelos = diferentes 'visões' do mesmo texto!")

## 💰 Impacto dos Tokens nos Custos

Tá, agora vem uma parte **MUITO** importante pra quem vai usar LLMs em produção: **CUSTOS**! 💸

### Por que tokens importam financeiramente?

- **APIs cobram por token**: GPT-4, Claude, etc.
- **Modelos têm limites**: Context window (janela de contexto)
- **Processamento custa**: Mais tokens = mais compute

### Exemplo de Custos (aproximados):
- **GPT-4**: ~$0.03 por 1K tokens (input)
- **GPT-3.5**: ~$0.002 por 1K tokens (input)  
- **Claude**: Varia entre $0.008-0.024 por 1K tokens

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-04_img_04.png)

**Dica do Pedro:** Sempre otimize seus prompts! Um prompt mal feito pode custar 3x mais que um otimizado. É tipo escolher entre Uber e ônibus! 🚌💰

In [ ]:
# Vamos criar uma calculadora de custos de tokens!

class CalculadoraCustoTokens:
    def __init__(self):
        # Preços aproximados por 1K tokens (USD) - Março 2024
        self.precos = {
            'GPT-4': {'input': 0.03, 'output': 0.06},
            'GPT-3.5-turbo': {'input': 0.0015, 'output': 0.002}, 
            'Claude-3-Opus': {'input': 0.015, 'output': 0.075},
            'Claude-3-Sonnet': {'input': 0.003, 'output': 0.015}
        }
    
    def calcular_custo(self, modelo, tokens_input, tokens_output=0):
        """Calcula o custo baseado no número de tokens"""
        if modelo not in self.precos:
            return None
            
        custo_input = (tokens_input / 1000) * self.precos[modelo]['input']
        custo_output = (tokens_output / 1000) * self.precos[modelo]['output']
        
        return {
            'custo_input': custo_input,
            'custo_output': custo_output,
            'custo_total': custo_input + custo_output
        }
    
    def comparar_modelos(self, tokens_input, tokens_output=0):
        """Compara custos entre diferentes modelos"""
        resultados = {}
        for modelo in self.precos.keys():
            resultados[modelo] = self.calcular_custo(modelo, tokens_input, tokens_output)
        return resultados

# Exemplo prático: analisando custos
calculadora = CalculadoraCustoTokens()

# Cenários de uso
cearios = [
    {"nome": "Prompt simples", "input": 50, "output": 100},
    {"nome": "Análise de documento", "input": 2000, "output": 500},
    {"nome": "Processamento em lote", "input": 10000, "output": 3000},
    {"nome": "Chatbot diário", "input": 50000, "output": 25000}
]

print("💰 Calculadora de Custos por Tokens\n")

for cenario in cearios:
    print(f"🎯 Cenário: {cenario['nome']}")
    print(f"   Input: {cenario['input']:,} tokens")
    print(f"   Output: {cenario['output']:,} tokens")
    print("   Custos por modelo:")
    
    comparacao = calculadora.comparar_modelos(cenario['input'], cenario['output'])
    
    for modelo, custos in comparacao.items():
        total = custos['custo_total']
        print(f"     {modelo}: ${total:.4f}")
    
    # Mostra o mais barato e mais caro
    custos_ordenados = sorted(comparacao.items(), key=lambda x: x[1]['custo_total'])
    mais_barato = custos_ordenados[0]
    mais_caro = custos_ordenados[-1]
    
    diferenca = mais_caro[1]['custo_total'] / mais_barato[1]['custo_total']
    
    print(f"   💡 Mais barato: {mais_barato[0]} (${mais_barato[1]['custo_total']:.4f})")
    print(f"   💸 Diferença: {diferenca:.1f}x mais caro que o mais barato\n")

print("📊 Dicas para otimizar custos:")
print("  • Use prompts concisos")
print("  • Reutilize contexto quando possível")
print("  • Escolha o modelo adequado para a tarefa")
print("  • Monitore uso de tokens em produção")

In [ ]:
# Vamos criar um gráfico de custos por modelo!

# Preparando dados para visualização
modelos = list(calculadora.precos.keys())
tokens_cenarios = [100, 500, 1000, 5000, 10000]

# Calcula custos para cada cenário (assumindo output = 50% do input)
custos_por_modelo = {modelo: [] for modelo in modelos}

for tokens in tokens_cenarios:
    tokens_output = tokens // 2  # 50% do input
    for modelo in modelos:
        custo = calculadora.calcular_custo(modelo, tokens, tokens_output)
        custos_por_modelo[modelo].append(custo['custo_total'])

# Cria o gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Custos por número de tokens
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
for i, (modelo, custos) in enumerate(custos_por_modelo.items()):
    ax1.plot(tokens_cenarios, custos, marker='o', linewidth=2.5, 
            label=modelo, color=colors[i], markersize=8)

ax1.set_xlabel('Número de Tokens (Input)')
ax1.set_ylabel('Custo Total (USD)')
ax1.set_title('💰 Custos por Modelo vs Número de Tokens')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Gráfico 2: Comparação de preços por 1K tokens
precos_input = [calculadora.precos[modelo]['input'] for modelo in modelos]
precos_output = [calculadora.precos[modelo]['output'] for modelo in modelos]

x_pos = np.arange(len(modelos))
width = 0.35

ax2.bar(x_pos - width/2, precos_input, width, label='Input', 
       color='lightblue', alpha=0.8)
ax2.bar(x_pos + width/2, precos_output, width, label='Output', 
       color='lightcoral', alpha=0.8)

ax2.set_xlabel('Modelos')
ax2.set_ylabel('Preço por 1K Tokens (USD)')
ax2.set_title('📊 Preços Input vs Output por Modelo')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([modelo.replace('-', '\n') for modelo in modelos], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Adiciona valores nas barras
for i, (inp, out) in enumerate(zip(precos_input, precos_output)):
    ax2.text(i - width/2, inp + 0.001, f'${inp:.3f}', 
            ha='center', va='bottom', fontweight='bold', fontsize=9)
    ax2.text(i + width/2, out + 0.001, f'${out:.3f}', 
            ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

# Insights
print("🎯 Insights dos gráficos:")
print("  • Custos crescem linearmente com tokens")
print("  • Output geralmente custa 2-5x mais que input")
print("  • GPT-3.5 é muito mais barato para tarefas simples")
print("  • Diferenças ficam enormes em grande escala!")

## 🚀 Exercício Prático 1: Analisador de Textos

**Desafio:** Crie um analisador que compara diferentes aspectos da tokenização!

### Sua missão:
1. Receber um texto qualquer
2. Analisar com 3 tokenizadores diferentes
3. Calcular estatísticas comparativas
4. Gerar um relatório visual

**Dica do Pedro:** Use tudo que aprendemos até agora! Seja criativo! 🎨

In [ ]:
# 🎯 EXERCÍCIO PRÁTICO 1: Analisador de Textos
# Complete o código abaixo!

class AnalisadorTokenizacao:
    def __init__(self):
        # TODO: Inicializar tokenizadores
        # self.bert_tokenizer = ???
        # self.gpt2_tokenizer = ???
        # (Use os tokenizadores já carregados)
        pass
    
    def analisar_texto(self, texto):
        """Analisa um texto com múltiplos tokenizadores"""
        # TODO: Implementar análise completa
        resultados = {
            'texto_original': texto,
            'caracteres': len(texto),
            'palavras': len(texto.split()),
            'bert': {
                # TODO: Preencher com análise BERT
                'tokens': None,
                'num_tokens': None,
                'tokens_especiais': None
            },
            'gpt2': {
                # TODO: Preencher com análise GPT-2
                'tokens': None,
                'num_tokens': None,
                'eficiencia': None  # tokens por palavra
            }
        }
        
        return resultados
    
    def gerar_relatorio(self, resultados):
        """Gera um relatório visual dos resultados"""
        # TODO: Criar visualizações
        # Dicas: gráfico de barras, estatísticas, comparações
        pass

# Seu código aqui!
# analisador = AnalisadorTokenizacao()
# resultado = analisador.analisar_texto("Sua frase de teste aqui!")
# analisador.gerar_relatorio(resultado)

print("🎯 Complete o código acima!")
print("💡 Dicas:")
print("  • Use os tokenizadores já carregados")
print("  • Calcule estatísticas interessantes")
print("  • Crie gráficos comparativos")
print("  • Seja criativo com as análises!")

## 🎲 Exercício Prático 2: Otimizador de Prompts

**Desafio Avançado:** Crie uma ferramenta que otimiza prompts para reduzir custos!

### Objetivos:
1. Receber um prompt "verboso"
2. Sugerir otimizações
3. Comparar custos antes/depois
4. Manter a qualidade do prompt

**Técnicas para usar:**
- Remoção de palavras desnecessárias
- Substituição por sinônimos mais curtos
- Reorganização da estrutura
- Uso de abreviações quando apropriado



In [ ]:
# 🎯 EXERCÍCIO PRÁTICO 2: Otimizador de Prompts
# Implemente as funções abaixo!

class OtimizadorPrompts:
    def __init__(self):
        # Dicionário de substituições para otimização
        self.substituicoes = {
            'por favor': 'pls',
            'você pode': 'pode',
            'muito obrigado': 'obrigado',
            'inteligência artificial': 'IA',
            'aprendizado de máquina': 'ML',
            'processamento de linguagem natural': 'NLP',
            # TODO: Adicione mais substituições!
        }
        
        # Palavras que podem ser removidas
        self.palavras_removiveis = [
            'realmente', 'absolutamente', 'definitivamente',
            'por favor', 'tipo', 'meio que'
            # TODO: Adicione mais palavras!
        ]
    
    def otimizar_prompt(self, prompt_original):
        """Otimiza um prompt para reduzir tokens"""
        # TODO: Implementar otimização
        # 1. Aplicar substituições
        # 2. Remover palavras desnecessárias
        # 3. Simplificar estruturas
        
        prompt_otimizado = prompt_original  # Placeholder
        
        return prompt_otimizado
    
    def comparar_custos(self, prompt_original, prompt_otimizado, modelo='GPT-4'):
        """Compara custos entre prompts"""
        # TODO: Usar a calculadora de custos
        # Calcular tokens e custos para ambos os prompts
        pass
    
    def gerar_relatorio_otimizacao(self, prompt_original, prompt_otimizado):
        """Gera relatório da otimização"""
        # TODO: Criar visualização comparativa
        pass

# Exemplo de uso:
prompt_teste = """
Olá! Por favor, você pode me ajudar a criar um texto sobre inteligência artificial? 
Eu realmente preciso de algo muito detalhado e definitivamente completo sobre 
aprendizado de máquina e processamento de linguagem natural. Muito obrigado!
"""

# Seu código aqui!
# otimizador = OtimizadorPrompts()
# prompt_otimizado = otimizador.otimizar_prompt(prompt_teste)
# otimizador.gerar_relatorio_otimizacao(prompt_teste, prompt_otimizado)

print("🎯 Implemente o otimizador de prompts!")
print("\n💡 Estratégias de otimização:")
print("  • Substituir termos longos por abreviações")
print("  • Remover palavras redundantes")
print("  • Usar estruturas mais concisas")
print("  • Manter o significado original")
print("\n🏆 Meta: Reduzir 30-50% dos tokens!")

## 🔗 Conexão com Próximos Módulos

Agora que você domina tokens, vamos ver como eles se conectam com o resto do curso!

### 🎯 Módulo 5 - Embeddings e Representações:
- **Tokens → Embeddings**: Como tokens viram vetores
- **Vocabulary embeddings**: Cada token tem sua representação
- **Dimensionalidade**: Geralmente 512, 768 ou 1024 dimensões

### 🎯 Módulo 6 - Tipos de Modelos:
- **Encoder-only** (BERT): Usa tokens especiais como [CLS], [SEP]
- **Decoder-only** (GPT): Gera tokens sequencialmente
- **Encoder-decoder** (T5): Combina ambos os approaches

### 🎯 Módulo 8 - Prompting:
- **Prompt engineering**: Otimizar tokens para melhores respostas
- **Context window**: Limite de tokens por interação
- **Token budgeting**: Gerenciar tokens em aplicações

**Dica do Pedro:** Tokens são a fundação de tudo! Entender bem esse módulo vai te ajudar muito nos próximos! 🏗️

In [ ]:
# Vamos criar uma prévia do que vem por aí!

print("🔮 Prévia dos Próximos Módulos\n")

# Simulando como tokens viram embeddings (Módulo 5)
print("🎯 Módulo 5 - De Tokens para Embeddings:")
texto_exemplo = "IA é incrível"
tokens = bert_tokenizer.tokenize(texto_exemplo)
print(f"  Tokens: {tokens}")
print(f"  Cada token vai virar um vetor de 768 dimensões!")
print(f"  Exemplo: 'IA' → [0.1, -0.3, 0.7, ..., 0.2] (768 números)")

# Context window (importante para Módulo 8)
print("\n🎯 Módulo 8 - Context Window:")
modelos_limits = {
    'GPT-3.5': '4K tokens',
    'GPT-4': '8K-32K tokens', 
    'Claude-3': '200K tokens',
    'Gemini-Pro': '1M tokens'
}

for modelo, limite in modelos_limits.items():
    print(f"  {modelo}: {limite}")

print("\n💡 Quanto mais tokens, mais 'memória' o modelo tem!")

# Tipos de modelos (Módulo 6)
print("\n🎯 Módulo 6 - Como diferentes arquiteturas usam tokens:")
print("  • BERT (Encoder): Vê todos os tokens simultaneamente")
print("  • GPT (Decoder): Gera um token por vez")
print("  • T5 (Encoder-Decoder): Combina ambas estratégias")

# Exemplo de geração sequencial
print("\n🤖 Como o GPT geraria: 'A IA vai'")
steps = [
    "Input: 'A IA vai' → Próximo token: 'revolucionar' (prob: 0.7)",
    "Input: 'A IA vai revolucionar' → Próximo token: 'o' (prob: 0.8)", 
    "Input: 'A IA vai revolucionar o' → Próximo token: 'mundo' (prob: 0.6)"
]

for i, step in enumerate(steps, 1):
    print(f"  {i}. {step}")
    
print("\n🚀 Muito mais vem por aí! Tokens são só o começo!")

## 🎉 Resumo do Módulo

**Liiindo! Você agora é um expert em tokens!** 🏆

### 🧠 O que aprendemos:

1. **Conceitos Fundamentais**:
   - O que são tokens e por que existem
   - Diferença entre caracteres, palavras e subwords
   - Importância da tokenização para LLMs

2. **Algoritmos de Tokenização**:
   - **BPE**: Usado pelo GPT
   - **WordPiece**: Usado pelo BERT
   - **SentencePiece**: Mais moderno e flexível

3. **Tokens Especiais**:
   - `<pad>`, `<unk>`, `<start>`, `<end>`
   - Como estruturam o processamento
   - Padding e attention masks

4. **Aspectos Práticos**:
   - **Custos**: Tokens = dinheiro em APIs
   - **Otimização**: Como reduzir tokens
   - **Context window**: Limites dos modelos

5. **Pipeline Completo**:
   - Texto → Tokens → IDs → Padding → Modelo
   - Cada etapa é crucial!

### 🔑 Pontos-chave para lembrar:

- **Diferentes modelos = diferentes tokenizações**
- **Mais tokens = mais custos**
- **Tokens especiais são fundamentais**
- **Otimização de prompts é essencial**

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-04_img_06.png)

### 🚀 Próximos passos:

No **Módulo 5**, vamos ver como esses tokens viram **embeddings** - representações numéricas que os modelos realmente entendem!

**Dica final do Pedro:** Tokens são a "linguagem" que você usa pra conversar com as LLMs. Dominar isso é tipo aprender o "dialeto" da IA! 🤖💬

---

**Parabéns por completar o Módulo 4!** 🎊

*Continue praticando com os exercícios e nos vemos no próximo módulo!*